In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn import metrics

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

In [ ]:
hdr10 = pd.read_csv(os.path.join(dirname, filename))

In [ ]:
hdr10.columns = hdr10.columns.str.lower()
hdr10.head()

In [ ]:
hdr10.set_index('gameid',inplace=True)

## **Exploratory Data Analysis**

In [ ]:
hdr10.info()

* The dataset has a total of 9879 rows and there seems to be no value missing for any attributes.
* There are a total of 38 attributes in the dataset.
* The Bluewins is the target variable, which we will be predicting

In [ ]:
hdr10.bluewins.value_counts()

* There is no imbalance in the wins and loss values of the target value

In [ ]:
#plt.figure(figsize=[15,15])
hdr10.hist(figsize=[15,18]);
#plt.show()

## **Feature Engineering**

Lets focus our analysis on few metrics of the game, which would help the blue team increase their chances of winning in the first 10 minutes of the game. We would be focusing on following metrics and see how they increase blue team's winning chances - 
* Number of minions or creeps killed more than the red team
* Number of chapions killed more than the red team
* Number of wards placed more than the red team

We don't have attributes which show the count difference between the above attributes for the 2 teams. So lets create these attributes in the given dataframe

In [ ]:
#creating minion kill difference attribute to show the difference in number of minions killed by blue and red team in a given game
hdr10['minionskilldifference'] = hdr10["bluetotalminionskilled"]-hdr10["redtotalminionskilled"]

#selecting rows where the blue team has killed more minions than the red team in first 10 mins
minionkillpositive_temp = hdr10.query("minionskilldifference>0")['bluewins']
minionkillpositive_pie = minionkillpositive_temp.value_counts()
minionkillpositive_pie.rename(index={1: "Won", 0:"Lost"}, inplace=True)

#selecting rows where the blue team has killed less minions than the red team in first 10 mins
minionkillnegative_temp = hdr10.query("minionskilldifference<=0")['bluewins']
minionkillnegative_pie = minionkillnegative_temp.value_counts()
minionkillnegative_pie.rename(index={1: "Won", 0:"Lost"}, inplace=True)

In [ ]:
fig = plt.figure(figsize=(10,8), dpi=100)
ax1 = plt.subplot2grid((2,1),(0,0))
plt.pie(minionkillpositive_pie, labels=minionkillpositive_pie.index, explode= (0,0.05),
       colors=['YellowGreen', 'Coral'],
       autopct='%1.1f%%', startangle=70)
plt.title("Blue Team Win Percentage with More Minion Kills than Red Team", fontsize=10, color='Black')

ax1 = plt.subplot2grid((2,1),(1,0))
plt.pie(minionkillnegative_pie, labels=minionkillnegative_pie.index, explode= (0,0.05),
       colors=['Coral', 'Yellowgreen'],
       autopct='%1.1f%%', startangle=70)
plt.title("Blue Team Win Percentage with Less Minion Kills than Red Team", fontsize=10, color='Black');

It's clear from the above charts that the blue team won 63% of the time where they had more minion kills in the first 10 minutes of the game.

However, there are 37.7% games where the blue team had less number of minion kills but still managed to win the game. 
It would be interesting to probe more in these games to see what strategy did the blue team adopt to win the game. Since this dataset has only first 10 minutes of the game data, our exploration scope is restricted and can't look at the team strategy in the later part of the game.

In [ ]:
#creating chanmpion kill difference attribute to show the difference in number of champions killed by blue and red team in a given game
hdr10["championkilldifference"] = hdr10["bluekills"] - hdr10["redkills"]

#selecting rows where the blue team has killed more champions than the red team in first 10 mins
champkillpositive_temp = hdr10.query("championkilldifference>0")['bluewins']
champkillpositive_pie = champkillpositive_temp.value_counts()
champkillpositive_pie.rename(index={1: "Won", 0:"Lost"}, inplace=True)

#selecting rows where the blue team has killed less champions than the red team in first 10 mins
champkillnegative_temp = hdr10.query("championkilldifference<=0")['bluewins']
champkillnegative_pie = champkillnegative_temp.value_counts()
champkillnegative_pie.rename(index={1: "Won", 0:"Lost"}, inplace=True)

In [ ]:
fig = plt.figure(figsize=(10,8), dpi=100)
ax1 = plt.subplot2grid((2,1),(0,0))
plt.pie(champkillpositive_pie, labels=champkillpositive_pie.index, explode= (0,0.05),
       colors=['YellowGreen', 'Coral'],
       autopct='%1.1f%%', startangle=70)
plt.title("Blue Team Win Percentage with More Champion Kills than Red Team in First 10 Minutes", fontsize=10, color='Black')

ax1 = plt.subplot2grid((2,1),(1,0))
plt.pie(champkillnegative_pie, labels=champkillnegative_pie.index, explode= (0,0.05),
       colors=['Coral', 'Yellowgreen'],
       autopct='%1.1f%%', startangle=70)
plt.title("Blue Team Win Percentage with Less Champion Kills than Red Team in First 10 Minutes", fontsize=10, color='Black');

It's evident from the above chart that the more champions you kill, the more you increase your chances of winning. 72.4% of the game were won by the blue team, where had more champion kills in the first 10 minutes of the game.

Again, there are exceptions, where 31.2% of the games were won by blue team even after having lesser number of champion kills than the red team

In [ ]:
hdr10['wardsdifference'] = hdr10["bluewardsplaced"]-hdr10["redwardsplaced"]

wardsdifferencepositive_temp = hdr10.query("wardsdifference>0")['bluewins']
wardsdifferencepositive_pie = wardsdifferencepositive_temp.value_counts()
wardsdifferencepositive_pie.rename(index={1: "Won", 0:"Lost"}, inplace=True)

wardsdifferencenegative_temp = hdr10.query("wardsdifference<=0")['bluewins']
wardsdifferencenegative_pie = wardsdifferencenegative_temp.value_counts()
wardsdifferencenegative_pie.rename(index={1: "Won", 0:"Lost"}, inplace=True)

In [ ]:
fig = plt.figure(figsize=(10,8), dpi=100)
ax1 = plt.subplot2grid((2,1),(0,0))
plt.pie(wardsdifferencepositive_pie, labels=wardsdifferencepositive_pie.index, explode= (0,0.05),
       colors=['YellowGreen', 'Coral'],
       autopct='%1.1f%%', startangle=70)
plt.title("Blue Team Win Percentage with More Wards Placed than Red Team in First 10 Minutes", fontsize=10, color='Black')

ax1 = plt.subplot2grid((2,1),(1,0))
plt.pie(wardsdifferencenegative_pie, labels=wardsdifferencenegative_pie.index, explode= (0,0.05),
       colors=['Coral', 'Yellowgreen'],
       autopct='%1.1f%%', startangle=70)
plt.title("Blue Team Win Percentage with Less Wards Placed than Red Team in First 10 Minutes", fontsize=10, color='Black');

### Feature Scaling and Selection

In [ ]:
hdr10_scaled = hdr10.copy()

# Create the scaler with object range of 0-1
scaler = MinMaxScaler()

# Fit and transform using the training data
hdr10_scaled[hdr10_scaled.columns] = scaler.fit_transform(hdr10[hdr10.columns])

In [ ]:
#Making series with all the independent features for the train dataset
features=hdr10_scaled.columns[1:]

In [ ]:
X_train_temp, X_test, y_train_temp, y_test = train_test_split(hdr10_scaled[features], 
                                                              hdr10_scaled["bluewins"], 
                                                              test_size=0.2, 
                                                              random_state=20)
# Perform the second split
X_train, X_valid, y_train, y_valid = train_test_split(X_train_temp, y_train_temp, 
                                                      test_size=0.25, random_state=20)

## Feature selection using lasso regression

In [ ]:
# Create the model
lasso = Lasso(alpha=0.01)

# Fit the model to the training data
lasso.fit(X_train, y_train)

# Extract the coefficients
lasso_coef = lasso.coef_

# Plot the coefficients
plt.figure(figsize=(20, 10))
plt.plot(range(len(X_train.columns)), lasso_coef)
plt.xticks(range(len(X_train.columns)), X_train.columns, rotation=60)
plt.axhline(0.0, linestyle='--', color='r')
plt.ylabel('Coefficients')

In [ ]:
temp = pd.DataFrame(X_train.columns.values, lasso_coef)
temp.reset_index(inplace=True)
temp.columns=['lasso_coef','features']
print(temp[temp['lasso_coef']!=0])

As per the above list of features selected by Lasso Regression, championkilldifference feature has been alloted the highest coefficient value showing that the it has a high impact on the team wins. This was also evident in our initial data exploration.

Some other interesting features that have come up are dragon kills and first blood. Let talk about these in detail as to why the model is considering these features would impact the winning chances of a team.

1. Dragons killed - Dragons are elite monsters found in the jungle. Depending on which dragon the team kills, they granted some power ups like increase attack or movement speed, etc. Our lasso regression model has selected this feature to impact the winning changes, which ideally makes sense. If the team secure early dragon kills in the game, they get different power ups which in turn help them kill more opposite team's champions and earn more gold.

2. Red First Blood - First bloods are the first champion killed by a team at the start of the game. In this case its Red First Blood and is negative, which means if the red team secures the first blood at the start of the game, it decreases the winning chances of the blue team.


In [ ]:
#selecting all the features with non-zero coefficient to train and test our model
lasso_selected_features = temp[(temp['lasso_coef']>0) | (temp['lasso_coef']<-0.052)]['features']
lasso_selected_features

In [ ]:
#Dataset with features selected by Lasso
lasso_Xtrain = X_train.loc[:,lasso_selected_features]
lasso_Xvalid = X_valid.loc[:,lasso_selected_features]
lasso_Xtest = X_test.loc[:,lasso_selected_features]

## **Prediction using Logistic Regression**

In [ ]:
#Training logistic regression model on features selected by lasso
lr_valid= LogisticRegression()
lr_valid.fit(lasso_Xtrain,y_train)
y_valid_pred = valid_lr.predict(lasso_Xvalid)

In [ ]:
cm_valid = confusion_matrix(y_valid,yvalid_pred)

In [ ]:
#creating confusion matrix for validation dataset prediction
plt.axes()
sns.heatmap(cm_valid/np.sum(cm_valid), annot=True, 
            fmt='.2%', cmap='Greens',
           xticklabels=['Loss','Wins'],
           yticklabels=['Loss','Wins'])
plt.xlabel("Predicted Label")
plt.ylabel("True Label");

In [ ]:
#checking accuracy of regression model on validation data set with features selected by lasso 
print('Accuracy of model on validation dataset: ',metrics.accuracy_score(y_valid, yvalid_pred))

In [ ]:
#Training logistic regression model on test data set to check the accuracy
lr_test= LogisticRegression()
lr_test.fit(lasso_Xtrain,y_train)
ytest_pred=lr_test.predict(lasso_Xtest)

In [ ]:
cm_test = confusion_matrix(y_test,ytest_predict)

In [ ]:
#creating confusion matrix for the test dataset prediction
plt.axes()
sns.heatmap(cm_test/np.sum(cm_test),annot=True,
           fmt='.2%', cmap='Greens',
           xticklabels=['Loss','Wins'],
           yticklabels=['Loss','Wins'])
plt.xlabel("Predicted Label")
plt.ylabel("True Label");

In [ ]:
print('Accuracy of model on test dataset :',metrics.accuracy_score(y_test,lasso_ypredtest))

Based on our model's prediction - features like getting the first champion kill, early dragon kills and killing more champions in the first 10 minutes of the game increases the chances of a team winning that game.

All these strategies are true based on the first 10 minutes of the game but may change drastically if we are able to analyze data which has stats for the entire duration of the game.